# TensorFlow Lite 模型生成
本次实验需按照教程完成基于TensorFlow Lite Model Maker的花卉
模型生成，并使用实验三的应用验证生成的模型。


## 预备工作
我们在Github的Codespaces中创建一个新的notebook，选择Python 3.8，然后输入如下代码安装程序运行必备的一些库。这里代码开头的“!”感叹号是告诉notebook把这一行的pip安装指令当做shell指令执行。

In [1]:
!pip install tflite-model-maker

  Using cached tflite_model_maker-0.4.3-py3-none-any.whl.metadata (5.4 kB)
  Using cached tf_models_official-2.3.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached numpy-1.23.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached pillow-10.3.0-cp38-cp38-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
  Using cached sentencepiece-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached tensorflow_datasets-4.9.2-py3-none-any.whl.metadata (9.0 kB)
  Using cached fire-0.6.0-py2.py3-none-any.whl
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl.metadata (41 kB)
  Using cached tflite_support-0.4.3-cp38-cp38-manylinux2014_x86_64.whl.metadata (2.4 kB)
  Using cached tensorflowjs-3.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached tensorflow-2.13.1-cp38-cp38-manylinux_2_17_

接下来，导入相关的库。

In [6]:
import os

import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

import matplotlib.pyplot as plt

在运行的过程中如果遇到了ImportError: libusb-1.0.so.0: cannot open shared object file: No such file的报错，我们需要安装libusb-1.0.so.0库。

In [1]:
!sudo apt-get install libusb-1.0-0 libusb-1.0-0-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libusb-1.0-0 is already the newest version (2:1.0.23-2build1).
libusb-1.0-0-dev is already the newest version (2:1.0.23-2build1).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


如果要我们升级ipywidgets，我们运行如下代码，然后在重新运行前面的导包操作。

In [2]:
%pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 4.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.7 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


## 模型训练

### 获取数据
本实验先从较小的数据集开始训练，当然越多的数据，模型精度更高。

这里从storage.googleapis.com中下载了本实验所需要的数据集。image_path可以定制，默认是在用户目录的.keras\datasets中。

In [7]:
image_path = tf.keras.utils.get_file(
      'flower_photos.tgz',
      'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
      extract=True)
image_path = os.path.join(os.path.dirname(image_path), 'flower_photos')

228827136/228813984 [==============================] - 1s 0us/step


### 运行示例
一共需4步完成。
第一步：加载数据集，并将数据集分为训练数据和测试数据。

In [8]:
data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

INFO:tensorflow:Load image with size: 3670, num_label: 5, labels: daisy, dandelion, roses, sunflowers, tulips.


2024-05-31 08:37:32.794436: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /workspaces/project-practice-3-091hzw/06_TensorFlow Lite 模型生成/.venv/lib/python3.8/site-packages/cv2/../../lib64:
2024-05-31 08:37:32.794468: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-31 08:37:32.794488: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-975054): /proc/driver/nvidia/version does not exist
2024-05-31 08:37:32.794746: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

第二步：训练Tensorflow模型

In [9]:
model = image_classifier.create(train_data)

INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hub_keras_layer_v1v2 (HubKe  (None, 1280)             3413024   
 rasLayerV1V2)                                                   
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 5)                 6405      
                                                                 
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________
None
Epoch 1/5


2024-05-31 08:38:23.130357: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-31 08:38:23.256218: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-31 08:38:23.313922: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-05-31 08:38:23.341011: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 154140672 exceeds 10% of free system memory.
2024-05-31 08:38:23.420760: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 57802752 exceeds 10% of free system memory.


103/103 [==============================] - 55s 520ms/step - loss: 0.8602 - accuracy: 0.7819
Epoch 2/5
103/103 [==============================] - 54s 519ms/step - loss: 0.6504 - accuracy: 0.9020
Epoch 3/5
103/103 [==============================] - 54s 521ms/step - loss: 0.6194 - accuracy: 0.9120
Epoch 4/5
103/103 [==============================] - 54s 524ms/step - loss: 0.6018 - accuracy: 0.9245
Epoch 5/5
103/103 [==============================] - 54s 519ms/step - loss: 0.5876 - accuracy: 0.9302


第三步：评估模型

In [10]:
loss, accuracy = model.evaluate(test_data)

12/12 [==============================] - 8s 488ms/step - loss: 0.6290 - accuracy: 0.9128


第四步，导出Tensorflow Lite模型

In [11]:
model.export(export_dir='.')

2024-05-31 08:44:11.416770: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmps007tmt9/assets


INFO:tensorflow:Assets written to: /tmp/tmps007tmt9/assets
2024-05-31 08:44:15.261460: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-05-31 08:44:15.261618: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2024-05-31 08:44:15.292601: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1164] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 913 nodes (656), 923 edges (664), time = 16.728ms.
  function_optimizer: function_optimizer did nothing. time = 0.015ms.

/workspaces/project-practice-3-091hzw/06_TensorFlow Lite 模型生成/.venv/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2024-05-31 08:44:16.177337: W tensorflow/compiler/mlir/lite/python/tf_tfl_

INFO:tensorflow:Label file is inside the TFLite model with metadata.


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 3
INFO:tensorflow:Label file is inside the TFLite model with metadata.


INFO:tensorflow:Saving labels in /tmp/tmpm7swx0yu/labels.txt


INFO:tensorflow:Saving labels in /tmp/tmpm7swx0yu/labels.txt


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


INFO:tensorflow:TensorFlow Lite model exported successfully: ./model.tflite


## 验证模型
这里导出的Tensorflow Lite模型包含了元数据(metadata),其能够提供标准的模型描述。导出的模型存放在Jupyter Notebook当前的工作目录中。我们把它下载到本地。

![image-20240531165625851](assets/image-20240531165625851.png)

然后重命名model.tflite为FlowerModel.tflite，然后打开上次的花卉识别项目，把FlowerModel.tflite放到ml目录中替换掉原有的模型。

![image-20240531170622572](assets/image-20240531170622572.png)

![image-20240531170800520](assets/image-20240531170800520.png)

然后在物理机上运行start项目，运行结果如下，可以看到运行正常，也能够识别出相应的花种

![image-20240531171149534](assets/image-20240531171149534.png)